In [1]:
#coding:utf-8

import tensorflow as tf
import numpy as np
import os
import time
import datetime
import data_helpers
from text_cnn import TextCNN
from tensorflow.contrib import learn

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected

In [2]:
# Parameters
# ==================================================

# Data loading params
# validation数据集占比
tf.flags.DEFINE_float("dev_sample_percentage", .1, "Percentage of the training data to use for validation")
# 正样本
tf.flags.DEFINE_string("positive_data_file", "data/rt-polaritydata/rt-polarity.pos", "Data source for the positive data.")
# 负样本
tf.flags.DEFINE_string("negative_data_file", "data/rt-polaritydata/rt-polarity.neg", "Data source for the negative data.")

# Model Hyperparameters
# 词向量长度
tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
# 卷积核大小
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
# 每一种卷积核个数
tf.flags.DEFINE_integer("num_filters", 64, "Number of filters per filter size (default: 128)")
# dropout参数
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
# l2正则化参数
tf.flags.DEFINE_float("l2_reg_lambda", 0.0005, "L2 regularization lambda (default: 0.0)")

# Training parameters
# 批次大小
tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
# 迭代周期
tf.flags.DEFINE_integer("num_epochs", 20, "Number of training epochs (default: 200)")
# 多少step测试一次
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
# 多少step保存一次模型
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
# 最多保存多少个模型
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
# tensorFlow 会自动选择一个存在并且支持的设备来运行 operation
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
# 获取你的 operations 和 Tensor 被指派到哪个设备上运行
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

# flags解析
tf.app.flags.DEFINE_string('f', '', 'kernel')
FLAGS = tf.flags.FLAGS
#FLAGS._parse_flags()

# 打印所有参数
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


Parameters:
ALLOW_SOFT_PLACEMENT=<absl.flags._flag.BooleanFlag object at 0x7f200a9c9358>
BATCH_SIZE=<absl.flags._flag.Flag object at 0x7f200a9c9128>
CHECKPOINT_EVERY=<absl.flags._flag.Flag object at 0x7f200a9c9320>
DEV_SAMPLE_PERCENTAGE=<absl.flags._flag.Flag object at 0x7f200a9b5c18>
DROPOUT_KEEP_PROB=<absl.flags._flag.Flag object at 0x7f200a9b5fd0>
EMBEDDING_DIM=<absl.flags._flag.Flag object at 0x7f200a9b5dd8>
EVALUATE_EVERY=<absl.flags._flag.Flag object at 0x7f200a9c9278>
F=<absl.flags._flag.Flag object at 0x7f200a9c9588>
FILTER_SIZES=<absl.flags._flag.Flag object at 0x7f200a9b5e80>
L2_REG_LAMBDA=<absl.flags._flag.Flag object at 0x7f200a9c90b8>
LOG_DEVICE_PLACEMENT=<absl.flags._flag.BooleanFlag object at 0x7f200a9c9400>
NEGATIVE_DATA_FILE=<absl.flags._flag.Flag object at 0x7f200a9b5d30>
NUM_CHECKPOINTS=<absl.flags._flag.Flag object at 0x7f200a9c93c8>
NUM_EPOCHS=<absl.flags._flag.Flag object at 0x7f200a9c91d0>
NUM_FILTERS=<absl.flags._flag.Flag object at 0x7f200a9b5f28>
POSITIVE_DAT

In [3]:
# Data Preparation
# ==================================================

# Load data
print("Loading data...")
x_text, y = data_helpers.load_data_and_labels(FLAGS.positive_data_file, FLAGS.negative_data_file)

# Build vocabulary
# 一行数据最多的词汇数
max_document_length = max([len(x.split(" ")) for x in x_text])
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
x = np.array(list(vocab_processor.fit_transform(x_text)))
print("x_shape:",x.shape)
print("y_shape:",y.shape)

# Randomly shuffle data
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = x[shuffle_indices]
y_shuffled = y[shuffle_indices]

# Split train/test set
# TODO: This is very crude, should use cross-validation
# 数据集切分为两部分
dev_sample_index = -1 * int(FLAGS.dev_sample_percentage * float(len(y)))
x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]
print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))

print("x:",x_train[0:5])
print("y:",y_train[0:5])

Loading data...
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
x_shape: (10662, 56)
y_shape: (10662, 2)
Vocabulary Size: 18758
Train/Dev split: 9596/1066
x: [[ 4719    59   182    34   190   804     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]
 [  129  7044   284   146    80     3  1116    58    84  1386   182  1968
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0   

In [4]:
# Training
# ==================================================

with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=y_train.shape[1],
            vocab_size=len(vocab_processor.vocabulary_),
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        # 计算梯度
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        # 将计算出的梯度应用到变量上，是函数minimize()的第二部分，
        # 返回一个应用指定的梯度的操作Operation，对global_step做自增操作
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        # 保存变量的梯度值
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        # 定义输出路径
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        # 保存模型，最多保存5个模型
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Write vocabulary
        vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            if (step%10==0):
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)

        # Generate batches
        batches = data_helpers.batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            # 测试
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            # 保存模型
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/hist is illegal; using conv-maxpool-4/W_0/grad/hist instead.
INFO:te

2019-02-22T21:31:29.323319: step 670, loss 0.640557, acc 0.65625
2019-02-22T21:31:29.823182: step 680, loss 0.526369, acc 0.765625
2019-02-22T21:31:30.342953: step 690, loss 0.583848, acc 0.71875
2019-02-22T21:31:30.851740: step 700, loss 0.567407, acc 0.734375

Evaluation:
2019-02-22T21:31:30.889022: step 700, loss 0.62307, acc 0.670732

Saved model checkpoint to /home/know/Documents/git/tensorflow/example-ben/cnn-text-classification-tf-master/runs/1550842240/checkpoints/model-700

2019-02-22T21:31:31.447923: step 710, loss 0.600454, acc 0.703125
2019-02-22T21:31:31.978410: step 720, loss 0.44204, acc 0.859375
2019-02-22T21:31:32.477791: step 730, loss 0.528428, acc 0.765625
2019-02-22T21:31:32.995885: step 740, loss 0.582717, acc 0.71875
2019-02-22T21:31:33.497512: step 750, loss 0.526894, acc 0.783333
2019-02-22T21:31:34.024923: step 760, loss 0.552281, acc 0.765625
2019-02-22T21:31:34.525873: step 770, loss 0.472144, acc 0.859375
2019-02-22T21:31:35.023581: step 780, loss 0.521038,

2019-02-22T21:32:18.709653: step 1610, loss 0.396068, acc 0.90625
2019-02-22T21:32:19.253786: step 1620, loss 0.367115, acc 0.9375
2019-02-22T21:32:19.760430: step 1630, loss 0.387602, acc 0.921875
2019-02-22T21:32:20.271205: step 1640, loss 0.401928, acc 0.90625
2019-02-22T21:32:20.807599: step 1650, loss 0.402216, acc 0.933333
2019-02-22T21:32:21.301832: step 1660, loss 0.338551, acc 0.984375
2019-02-22T21:32:21.799523: step 1670, loss 0.42009, acc 0.890625
2019-02-22T21:32:22.295691: step 1680, loss 0.396887, acc 0.90625
2019-02-22T21:32:22.788922: step 1690, loss 0.370227, acc 0.953125
2019-02-22T21:32:23.288178: step 1700, loss 0.368054, acc 0.953125

Evaluation:
2019-02-22T21:32:23.323278: step 1700, loss 0.569434, acc 0.72045

Saved model checkpoint to /home/know/Documents/git/tensorflow/example-ben/cnn-text-classification-tf-master/runs/1550842240/checkpoints/model-1700

2019-02-22T21:32:23.892599: step 1710, loss 0.457782, acc 0.84375
2019-02-22T21:32:24.394492: step 1720, los

2019-02-22T21:33:07.383105: step 2540, loss 0.359795, acc 0.953125
2019-02-22T21:33:07.889260: step 2550, loss 0.336431, acc 0.983333
2019-02-22T21:33:08.389147: step 2560, loss 0.354877, acc 0.953125
2019-02-22T21:33:08.911681: step 2570, loss 0.355446, acc 0.953125
2019-02-22T21:33:09.415397: step 2580, loss 0.346708, acc 0.96875
2019-02-22T21:33:09.930268: step 2590, loss 0.387826, acc 0.921875
2019-02-22T21:33:10.447759: step 2600, loss 0.321405, acc 1

Evaluation:
2019-02-22T21:33:10.484637: step 2600, loss 0.561598, acc 0.745779

Saved model checkpoint to /home/know/Documents/git/tensorflow/example-ben/cnn-text-classification-tf-master/runs/1550842240/checkpoints/model-2600

2019-02-22T21:33:11.088903: step 2610, loss 0.332135, acc 0.984375
2019-02-22T21:33:11.613270: step 2620, loss 0.365221, acc 0.953125
2019-02-22T21:33:12.133591: step 2630, loss 0.359273, acc 0.953125
2019-02-22T21:33:12.655463: step 2640, loss 0.328552, acc 0.984375
2019-02-22T21:33:13.173894: step 2650, los